In [1]:
import pandas as pd
from src.train import train_loop, test_loop
from src.model import CNNLSTMModel
from src.dataset import InverterTimeSeriesDataset
from src.preprocess import *
import torch

inverter_data = load_parquet_data('data/inverter_data')
failure_sessions = load_failure_sessions('data/failure_sessions.csv', min_days=3)

Loaded 15 parquet files → 6126272 rows
Kept 61 sessions longer than 3 days


In [2]:
labeled_df = prepare_dataset(inverter_data, failure_sessions)
labeled_df.fillna(0, inplace=True)

Total pre-failure rows: 82486
Total rows: 5905370


In [3]:
train_df, test_df = train_test_split_on_time(labeled_df, 0.2)
val_df, test_df = train_test_split_on_time(test_df, 0.5)

Train set size: 4724296 Train set time range: 2021-12-02 00:00:00 to 2024-10-27 01:00:00
Test set size: 1181074 Test set time range: 2024-10-27 01:00:00 to 2025-07-23 23:35:00
Train set size: 590537 Train set time range: 2024-10-27 01:00:00 to 2025-03-04 04:40:00
Test set size: 590537 Test set time range: 2025-03-04 04:40:00 to 2025-07-23 23:35:00


In [4]:
feature_cols = [
    "metric.AC_POWER.MEASURED",
    "metric.DC_POWER.MEASURED",
    "metric.AC_CURRENT_A.MEASURED",
    "metric.AC_CURRENT_B.MEASURED",
    "metric.AC_CURRENT_C.MEASURED",
    "metric.DC_CURRENT.MEASURED",
    "metric.DC_CURRENT_AVG.MEASURED",
    "metric.DC_CURRENT_MAX.MEASURED",
    "metric.FREQUENCY.MEASURED",
    "metric.POWER_FACTOR.MEASURED",
    "metric.HEARTBEAT.MEASURED",
    "metric.COMM_LINK.MEASURED",
    "metric.STATUS_WARNING_WORD.MEASURED",
    "metric.STATUS_FAULT_WORD.MEASURED",
    "metric.STATUS_IGBT_MAX_TEMP.MEASURED",
    "metric.STATUS_INTERNAL_TEMP.MEASURED",
    "metric.STATUS_INTERNAL_HUMIDITY.MEASURED"
]


from torch.utils.data import DataLoader

train_ds = InverterTimeSeriesDataset(train_df, feature_cols, under_sample=True)
val_ds   = InverterTimeSeriesDataset(val_df,   feature_cols)
test_ds  = InverterTimeSeriesDataset(test_df,  feature_cols)

Processing devices: 100%|██████████| 16/16 [00:01<00:00,  8.40it/s]


In [5]:
batch_size = 2**13
train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True,  num_workers=6, pin_memory=True)
val_loader   = DataLoader(val_ds,   batch_size=batch_size, shuffle=False, num_workers=6, pin_memory=True)
test_loader  = DataLoader(test_ds,  batch_size=batch_size, shuffle=False, num_workers=6, pin_memory=True)

In [6]:
model = CNNLSTMModel(num_features=len(feature_cols))
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = torch.nn.MSELoss()

In [7]:
train_loop(model, train_loader, val_loader, log_interval=10, num_epochs=10, optimizer=optimizer, criterion=criterion)

Model moved to cuda
[Epoch 1/10] Step 0/16 - Loss: 0.2510
[Epoch 1/10] Step 10/16 - Loss: 0.2490
🔁 Epoch 1 finished. Avg Train Loss: 0.2491
✅ Validation Loss: 0.2291 | Accuracy: 94.95%
[Epoch 2/10] Step 0/16 - Loss: 0.2474
[Epoch 2/10] Step 10/16 - Loss: 0.2452
🔁 Epoch 2 finished. Avg Train Loss: 0.2456
✅ Validation Loss: 0.2293 | Accuracy: 75.31%
[Epoch 3/10] Step 0/16 - Loss: 0.2433
[Epoch 3/10] Step 10/16 - Loss: 0.2422
🔁 Epoch 3 finished. Avg Train Loss: 0.2412
✅ Validation Loss: 0.2258 | Accuracy: 70.57%
[Epoch 4/10] Step 0/16 - Loss: 0.2398
[Epoch 4/10] Step 10/16 - Loss: 0.2364
🔁 Epoch 4 finished. Avg Train Loss: 0.2377
✅ Validation Loss: 0.2297 | Accuracy: 72.89%
[Epoch 5/10] Step 0/16 - Loss: 0.2360
[Epoch 5/10] Step 10/16 - Loss: 0.2337
🔁 Epoch 5 finished. Avg Train Loss: 0.2348
✅ Validation Loss: 0.2458 | Accuracy: 64.02%
[Epoch 6/10] Step 0/16 - Loss: 0.2355
[Epoch 6/10] Step 10/16 - Loss: 0.2320
🔁 Epoch 6 finished. Avg Train Loss: 0.2329
✅ Validation Loss: 0.2352 | Accurac

In [8]:
predictions = test_loop(model, test_loader, device='cuda', criterion=criterion)

🔍 Test Loss: 0.2389 | Accuracy: 55.47%


In [14]:
pd.value_counts(predictions)

False    332281
True     256951
dtype: int64

In [16]:
from sklearn.metrics import classification_report, confusion_matrix
print(classification_report(test_ds.y.numpy(), predictions, target_names=['Normal', 'Failure']))
print(confusion_matrix(test_ds.y.numpy(), predictions))

              precision    recall  f1-score   support

      Normal       0.97      0.56      0.71    571737
     Failure       0.02      0.34      0.04     17495

    accuracy                           0.55    589232
   macro avg       0.49      0.45      0.38    589232
weighted avg       0.94      0.55      0.69    589232

[[320812 250925]
 [ 11469   6026]]
